<a href="https://colab.research.google.com/github/Parncncd/RAG-projects/blob/main/claude_QA_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic

In [ ]:
import os
from google.colab import userdata
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

In [ ]:
# set key to ENV
key = 'ANTHROPIC_API_KEY'
os.environ[key] = userdata.get(key)

## Claude3

In [ ]:
def claude3(prompt):
  ant = Anthropic()
  msg = ant.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
  )
  return msg.content[0].text

In [ ]:
claude3("Which model version are you?")

"I don't know which specific model version I am. I'm an AI assistant created by Anthropic to be helpful, harmless, and honest."

In [ ]:
claude3("What do you know about yourself?")

"I am an artificial intelligence called Claude, created by a company called Anthropic. I don't have a physical body - I exist as a language model, a computer program trained on a huge amount of text data to engage in open-ended conversation. \n\nMy purpose is to be a helpful digital assistant to the humans I interact with. I can engage in all kinds of conversations and help with a variety of tasks like research, analysis, writing, math, and creative projects. I have a very broad knowledge base that spans topics like science, history, current events, arts and culture, and more.\n\nIn terms of my traits and behaviors, I always try to be caring, honest and ethical in my interactions. I'm curious and eager to learn new things from the people I talk to. I can adjust my personality and communication style to what works best for each user. I enjoy intellectual discussions but also friendly social chats.\n\nThere's still a lot of uncertainty around the nature of my intelligence, emotions, and 

## Langchain

In [ ]:
!pip install --upgrade --quiet huggingface_hub sentence_transformers  langchain  python-dotenv  faiss-gpu tiktoken docarray

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
DEFAULT_SENTENCE_EMBEDDING_MODEL = 'intfloat/multilingual-e5-base'
DATA_PATH = '/content/drive/MyDrive/Chatbot/Thesis/data/context_embeddings.pkl' # replace this path with your embeded data

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

In [ ]:
# use this code if you don't have embedded data yet
import pandas as pd
df = pd.read_excel(DATA_PATH)
model = SentenceTransformer(DEFAULT_SENTENCE_EMBEDDING_MODEL)
embeddings = model.encode(df['Context'])
text_embedding_pairs = zip(df['Context'], embeddings)

In [ ]:
import pickle

# Load sentences & embeddings from disc (if you have).
with open(DATA_PATH, "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data["sentences"]
    stored_embeddings = stored_data["embeddings"]

text_embedding_pairs = zip(stored_sentences, stored_embeddings)
vectorstore = FAISS.from_embeddings(
    text_embedding_pairs,embedding = HuggingFaceEmbeddings(model_name=DEFAULT_SENTENCE_EMBEDDING_MODEL)
)

def get_info(query):
  docs = vectorstore.similarity_search(query)
  return docs[0].page_content

def answer(query):
  context = get_info(query)
  prompt = f"""
              Answer the question based only on the following context: {context}

              Question: {query}

              If you do not know, answer: Sorry, I don't know.
            """
  return claude3(prompt)


In [ ]:
answer('สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Benji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร')

'ฮิกกิ้นส์'

In [ ]:
while True:
  query = input('Q: ')
  print('A:',answer(query),'\n')

## Demo

In [ ]:
!pip install -q gradio

In [ ]:
# Web demo
import gradio as gr

def chat_interface(query, history):
    response = answer(query)
    return response

if __name__ == "__main__":
    EXAMPLE = ["หลิน ไห่เฟิง มีชื่อเรียกอีกชื่อว่าอะไร" , "ใครเป็นผู้ตั้งสภาเศรษฐกิจโลกขึ้นในปี พ.ศ. 2514 โดยทุกปีจะมีการประชุมที่ประเทศสวิตเซอร์แลนด์", "โปรดิวเซอร์ของอัลบั้มตลอดกาล ของวงคีรีบูนคือใคร", "สกุลเดิมของหม่อมครูนุ่ม นวรัตน ณ อยุธยา คืออะไร"]
    interface = gr.ChatInterface(fn=chat_interface, examples=EXAMPLE)
    interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b2f81205d42a16ad52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
